# 기업 주요 분석 과제 구현

## 0. Import

In [ ]:
import os
from dotenv import load_dotenv
import datetime

import pandas as pd
import numpy as np

import plotly.graph_objects as go

from selenium import webdriver
from selenium.webdriver.common.by import By

from typing import Annotated, List
from typing_extensions import TypedDict

import datetime

from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage

from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent

In [2]:
load_dotenv()

True

## 1. 경쟁 기업 개요/현황

### 1-1. 기본 정보/규모

> 설립 연도, 위치, 조직 규모, 대표 이사, 주 사업 분야 등

- 잡코리아

In [111]:
class BaseInfoLoader:
    def __init__(self, url):
        self.url = url
        self.contents = []
    
    def load(self, in_pandas=False):
        # search url
        driver = webdriver.Chrome()
        driver.get(self.url)

        # find element with xpath
        elements = driver.find_elements(By.XPATH, '//*[@id="company-body"]/div[1]/div[1]/div/table')
        for element in elements:
            self.contents.append(element.text)
        
        # create base info dict
        keys = []
        values = []        
        for idx, text in enumerate(self.contents[0].split('\n')[1:]):
            keys.append(text) if idx%2==0 else values.append(text)
        self.info_dict = {
            key : value
            for key, value in zip(keys, values)
        }

        if in_pandas:
            info_df = pd.DataFrame([self.info_dict])
            return info_df
        else:
            return self.info_dict


In [112]:
url = "https://www.jobkorea.co.kr/Company/1696583"

loader = BaseInfoLoader(url)

In [ ]:
info_df = loader.load(in_pandas=True)

info_df

In [ ]:
loader.info_dict

{'산업': '이동전화기 제조업',
 '사원수': '129,095명 (2024.12.31)',
 '기업구분': '대기업',
 '설립일': '1969.12.01 (57년차)',
 '자본금': '8,975억 1천만원 (2024.12.31)',
 '매출액': '209조 5백억원 (2024.12.31)',
 '대표자': '전영현',
 '대졸초임': '5,163만원 상세보기',
 '주요사업': '휴대폰,컴퓨터,네트워크시스템,핵심칩,반도체부품,디스플레이패널,가전제품,의료기기,프린터 제조',
 '4대보험': '국민연금, 건강보험, 고용보험, 산재보험',
 '홈페이지': 'http://www.samsung.com/sec',
 '주소': '경기 수원시 영통구 삼성로 129 (매탄동, 삼성전자)',
 '계열사': '삼성그룹 스테코(주) 삼성전자판매(주) 삼성벤처투자(주) 삼성전자로지텍(주) (주)시큐아이 (주)씨브이네트 (주)멀티캠퍼스 삼성생명서비스손해사정(주) (주)미라콤아이앤씨 세메스(주) 에스디플렉스(주) 삼성중공업(주) 삼성SDI(주) 삼성전기(주) 삼성메디슨(주) (주)하만인터내셔널코리아 더보기'}

### 1-2. 비즈니스 모델


비즈니스 모델의 핵심 구성 요소:

- 고객 관계: 지속적인 관계를 조성하고 시간이 지남에 따라 고객 충성도를 구축하는 방법에 중점
- 가치 제안: 제품 또는 서비스를 독특하고 매력적이며 고객에게 가치 있는 것으로 만드는 요소를 정의
- 목표 고객: 이상적인 고객과 그들의 가장 시급한 니즈를 파악
- 수익 흐름: 비즈니스가 어떻게 수입을 창출하는지 설명
- 비용 구조: 비즈니스를 이끄는 주요 운영 비용을 강조
- 핵심 리소스: 성공에 중요한 물리적, 인적, 재정적 또는 지적 재산과 같은 필수 자산에 대해 설명
- 배포 채널: 제품 또는 서비스를 고객과 연결하고 고객에게 제공하는 방법을 설명

> 이후 분석 과제에서 '고객층', '차별화 요소' 에 대한 분석이 이루어지므로,

> '수익 흐름', '비용 구조' 를 중심으로 시각화 하는 것이 좋을 듯
>> 이를 위해서는 '재무제표'활용 필요

In [ ]:
labels = [
    "DX부문 매출", "DS부문 매출", "SDC", "Harman", "기타", # 1단계
    "총 매출",                                         # 2단계
    "매출총이익", "매출원가",                             # 3단계 
    "영업이익", "판매비와관리비",                              # 4단계
    "순이익", "법인세", "판매비와관리비", "기타비용"           # 5단계
]

# 단계별로 source와 target을 연결
source = [0, 1, 2, 3, 4, 5, 5, 6, 6, 8, 8, 9, 9]
target = [5, 5, 5, 5, 5, 6, 7, 8, 9, 10, 11, 12, 13]
values = [1748877, 1110660, 291578, 142749, 285155, 
          1143086, 1865622, 
          327259, 815826, 
          815826, 327259]

fig = go.Figure(go.Sankey(
    node=dict(
        pad=15, thickness=20,
        line=dict(color="black", width=0.5),
        label=labels,
    ),
    link=dict(
        source=source,
        target=target,
        value=values
    )
))

fig.update_layout(title_text="삼성전자 수익구조", font_size=10)

### 1-3. 기업 평가/인식 분석

> 뉴스, 보도자료, 리뷰 등 인터넷 자료 수집 및 분석
>> 워드클라우드, 긍/부정, 네트워크분석 등 ...

- 기업 평가 및 인식
    1. 기업 브랜드 인식
    2. 제품 인식

In [28]:
"""
> tavily search result schema

{
    'title' : ,
    'url' : ,
    'content' : ,
    'score' : ,
}
"""

tavily_search_tool = TavilySearchResults(
    max_results=5,
    search_depth="advanced"
)

llm = ChatOpenAI(
    temperature=0,
    model='gpt-4o'
)

In [29]:
@tool
def web_search_tavily(query):
    """
    Web search tool using TavliySearchResult class.

    Attributes:
        query : search query
    
    Returns:
        web_search_results (List[dict]) : list of search results
        urls : source urls of search results
    """
    # get web search result
    search_results = tavily_search_tool.invoke(query)

    # get url list
    urls = []
    for search_result in search_results:
        urls.append(search_result['url'])
    urls = list(np.unique(urls))

    return {
        "web_search_results" : search_results,
        "urls" : urls
    }

In [87]:
def search_agent_node(state):
    company = state['company']

    today = datetime.datetime.today()
    search_prompt = SystemMessage(
        content=f"""당신은 전문적인 기업 인식 조사관입니다. 오늘은 {today} 입니다.
    당신의 임무는 특정 기업 및 기업의 제품에 대한 최근 대중의 반응과 인식에 대해 검색하고 그 결과를 수집하는 것입니다.
    당신이 수집할 기업: {company}

    다음의 지침에 따라 사용자가 원하는 기업에 대한 인식/반응 자료 수집 결과를 반환하세요.
    <지침>
        1. '뉴스 및 기사', '제품 리뷰', 'SNS 반응', '관련 영상'의 분류에 입각하여 대중의 반응과 인식을 수집할 것.
        2. 기업의 이슈를 정리하는 것이 아닌, 대중의 반응을 확인할 수 있는 자료를 수집할 것.
        3. 검색 결과를 확인하여 기업에 대한 반응/인식을 명확히 알 수 없을 경우 해당 자료를 제외하고 다시 검색을 수행할 것.
        4. 수집한 자료를 목록화 하여 제공하되, 각 자료 별 '출처'/'내용'/'대중의 반응' 을 정리해서 제공할 것. 반드시 해당 자료의 출처를 함꼐 명시할 것.
        5. 수집된 대중의 반응을 요약하여 함께 제공할 것.
        6. 긍정적인 반응과 부정적인 반응 모두 포함할 것.
    </지침>
    """
    )

    search_agent = create_react_agent(
        model=llm,
        tools=[web_search_tavily],
        prompt=search_prompt
    )

    response = search_agent.invoke(state)

    state["response"] = response

    return state

In [88]:
class State(TypedDict):
    company: str
    messages: Annotated[list, add_messages]
    response: str
    urls: List[str]

In [89]:
workflow = StateGraph(State)

workflow.add_node("search_agent_node", search_agent_node)

workflow.add_edge(START, "search_agent_node")
workflow.add_edge("search_agent_node", END)

app = workflow.compile()

In [90]:
config = {
    "recursion_limit" : 50
}

input = {
    "company" : "삼성전자",
    "response" : "",
    "urls" : []
}

response = app.invoke(
    input=input,
    config=config
)

In [91]:
print(response["response"]["messages"][-1].content)

다음은 삼성전자에 대한 최근 대중의 반응과 인식을 수집한 결과입니다.

### 뉴스 및 기사
1. **출처**: [삼성전자, 2025년 1분기 실적 발표](https://news.samsungsemiconductor.com/kr/%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-2025%EB%85%84-1%EB%B6%84%EA%B8%B0-%EC%8B%A4%EC%A0%81-%EB%B0%9C%ED%91%9C/)
   - **내용**: 삼성전자는 2025년 1분기 매출 79.14조원, 영업이익 6.7조원을 기록하며 전분기 대비 매출이 4% 증가했다고 발표했습니다.
   - **대중의 반응**: 대체로 긍정적이며, 삼성전자의 지속적인 성장과 기술 혁신에 대한 기대가 높습니다.

2. **출처**: [CES 2025 - Samsung Newsroom](https://news.samsung.com/kr/ces-2025)
   - **내용**: 삼성전자는 CES 2025에서 다양한 혁신 제품을 선보이며 주목을 받았습니다.
   - **대중의 반응**: 혁신적인 제품에 대한 긍정적인 반응이 많으며, 특히 AI 기술에 대한 관심이 높습니다.

### 제품 리뷰
1. **출처**: [TV 추천 : 삼성전자 TOP 6 리뷰&비교 (2025년) - 노서치](https://nosearch.com/recommendation/pick/living/tv)
   - **내용**: 삼성전자의 QLED TV가 뛰어난 화질과 합리적인 가격으로 추천되었습니다.
   - **대중의 반응**: 긍정적이며, 특히 화질과 가격 대비 성능에 대한 만족도가 높습니다.

2. **출처**: [2025년 삼성 가전 신제품 총정리 – 혁신과 스마트함이 살아있는 ...](https://morningstar7.tistory.com/467)
   - **내용**: 삼성전자의 신제품들이 친환경적이고 스마트한 기능을 강조하고 있습니다.
   - **대중의 반응**: 긍정적이며, 환경 친화적인 제품에 대한 

- 수집된 링크를 크롤링하여 인식/반응 데이터셋 제작
    - 이를 기반으로 감정 분석, 워드 클라우드, 네트워크 분석 등 시행 가능

## 2. 시장 분석

### 2-1. 시장 현황